## 实验介绍

### 1.实验内容

本实验介绍协同过滤算法。

### 2.实验目标

通过本实验掌握协同过滤算法。

### 3.实验知识点

* 协同过滤算法

### 4.实验环境

* python 3.6.5  
* CourseGrading在线实验环境

### 5.预备知识

* 初等数学知识  
* Linux命令基本操作  
* Python编程基础

## 基于协同过滤的推荐引擎 
目前，我们接触到的比如网易云音乐的个性化推荐歌曲，淘宝的推荐物品，亚马逊的推荐图书等等。推荐引擎已经是一个很普遍的事物了。那么如何实现推荐功能呢？本节我们介绍一种协同过滤(collaborative filtering)的方法。协同过滤是通过将用户和其他用户的数据进行对比来实现推荐的。

协同过滤算法通过对用户历史行为数据的挖掘发现用户的偏好，基于不同的偏好对用户进行群组划分并推荐品味相似的商品。协同过滤推荐算法分为两类，分别是基于用户的协同过滤算法(user-based collaboratIve filtering)，和基于物品的协同过滤算法(item-based collaborative filtering)。简单的说就是：人以类聚，物以群分。

协同过滤与领域无关，它不需要知道现在对什么评分，谁在评分，评分是多少。而是基于各个类别之间的相似度来进行推荐。

这里涉及到了相似度计算的问题。推荐引擎如何决定将哪个物品推荐给用户，就是取决于计算出的相似度排序。

## 相似度计算 
我们希望找到一种计算物品之间的相似度的定量的方法。利用用户对物品的评分来计算相似度，这就是协同过滤中使用的方法。

下图中，给出了由一些用户及其对部分菜肴的评级信息所组成的矩阵。
![](coll_filter/1_coll_filter.png)
下面，我们基于所给数据矩阵，介绍三种相似度计算方法：
### 1.欧式距离
欧几里德距离评价是一个较为简单的用户关系评价方法。原理是通过计算两个用户在散点图中的距离来判断不同的用户是否有相同的偏好。我们计算一下手撕猪肉和烤牛肉之间的相似度。一开始我们使用欧氏距离来计算。手撕猪肉和烤牛肉的欧氏距离为：
![](coll_filter/2_coll_filter.png)
而手撕猪肉和鳗鱼饭的欧氏距离为：
![](coll_filter/3_coll_filter.png)
在该数据中，由于手撕猪肉和烤牛肉的距离小于手撕猪肉和鳗鱼饭的距离，因此手撕猪肉与烤牛肉比与鳗鱼饭更为相似。我们希望，相似度值在0到1之间变化，并且物品对越相似，它们的相似度值也就越大。我们可以用“相似度=1/(1+距离)”这样的算式来计算相似度。当距离为0时，相似度为1.0。如果距离真的非常大时，相似度也就趋近于0。
### 2.皮尔逊相关系数(pearson correlation)
它度量两个向量之间的相似度。该方法相对于欧氏距离的一个优势在于，它对用户评级的量级并不敏感。比如某个狂躁者对所有物品的评分都是5分，而另一个忧郁者对所有物品的评分都是1分，皮尔逊相关系数会认为这两个向量是相等的。在NumPy中，皮尔逊相关系数的计算是由函数corrcoef()进行的，后面我们很快就会用到它了。皮尔逊相关系数的取值范围从-1到+1，我们通过0.5 + 0.5*corrcoef()这个函数计算，并且把其取值范围归一化到0到1之间。
### 3.余弦相似度(cosine similarity)
计算的是两个向量夹角的余弦值。如果夹角为90度，则相似度为0；如果两个向量的方向相同，则相似度为1.0。同皮尔逊相关系数一样，余弦相似度的取值范围也在-1到+1之间，因此我们也将它归一化到0到1之间。计算余弦相似度值，我们采用的两个向量A和B夹角的余弦相似度的定义如下：
![](coll_filter/4_coll_filter.png)
接下来我们将上述各种相似度的计算方法写成Python中的函数。添加以下代码：

In [4]:
from numpy import *
import matplotlib.pyplot as plt
from numpy import linalg as la

def loadExData1():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]
"""
函数说明：相似度计算函数(欧氏距离)
parameters：
    inA -列向量A
    inB -列向量B
return：
    两个向量的相似度
"""
def ecludSim(inA, inB):
    return 1.0/(1.0+la.norm(inA-inB))
"""
函数说明：相似度计算函数(皮尔逊相关系数)
parameters：
    inA -列向量A
    inB -列向量B
return：
    两个向量的相似度
"""
def pearsSim(inA, inB):
    if len(inA)<3: return 1.0   #是否存在三个或更多的点。两个向量是完全相关的，返回1
    #print(corrcoef(inA,inB,rowvar=0))
    return 0.5+0.5*corrcoef(inA,inB,rowvar=0)[0][1]     #将数据归一化到0到1之间
"""
函数说明：相似度计算函数(余弦相似度)
parameters：
    inA -列向量A
    inB -列向量B
return：
    两个向量的相似度
"""
def cosSim(inA, inB):
    num = float(inA.T*inB)  #计算分子
    denom = la.norm(inA)*la.norm(inB)   #计算分母
    return 0.5+0.5*(num/denom)  #归一化

if __name__ == '__main__':
    #测试相似度计算
    myMat = mat(loadExData1())
    print("欧氏距离：")
    print(ecludSim(myMat[:,0],myMat[:,4]))
    print(ecludSim(myMat[:,0],myMat[:,0]))
    print('皮尔逊相关系数：')
    print(pearsSim(myMat[:,0],myMat[:,4]))
    print(pearsSim(myMat[:,0],myMat[:,0]))
    print('余弦相似度：')
    print(cosSim(myMat[:,0],myMat[:,4]))
    print(cosSim(myMat[:,0],myMat[:,0]))

欧氏距离：
0.129731907557
1.0
皮尔逊相关系数：
0.205965381738
1.0
余弦相似度：
0.5
1.0


运行后，我们可以看到对于物品0和物品4的相似度计算，以及物品0与自身的相似度计算，肯定是1啦！要注意的是，上述计算中都是假设数据采用了列向量方式来进行计算的。这里的列向量也暗示了我们将利用基于物品的相似度计算方法。

##  基于物品的相似度还是用户的相似度 
在上述计算中，我们是基于物品(item-based)的相似度。另一种计算方法为基于用户(user-based)的相似度，用来计算用户距离。在图中矩阵也可以看出，行为用户，列为物品。到底使用哪一种相似度呢？这取决于用户或物品的数目。基于物品相似度计算的时间会随物品数量的增加而增加，基于用户的相似度计算的时间则会随用户数量的增加而增加。如果我们有一个商店，那么最多会有几千件商品。如果用户的数目很多，那么我们可能倾向于使用基于物品相似度的计算方法。对于大部分产品导向的推荐引擎而言，用户的数量往大于物品的数量，即购买商品的用户数会多于出售的商品种类。

## 推荐引擎的评价 
如何对推荐引擎进行评价呢？此时，我们既没有预测的目标值，也没有用户来调查他们对预测的满意程度。这里我们就可以采用前面多次使用的交叉测试的方法。通常用于推荐引擎评价的指标是称为最小均方根误差（Root Mean Squared Error，RMSE）的指标，它首先计算均方误差的平均值然后取其平方根。

## 参考文献与延伸阅读

### 参考资料:

1.哈林顿，李锐. 机器学习实战 : Machine learning in action[M]. 人民邮电出版社, 2013.  
2.周志华. 机器学习:Machine learning[M]. 清华大学出版社, 2016.

### 延伸阅读

1.李航. 统计学习方法[M]. 清华大学出版社, 2012.